# Classification

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import ensemble, model_selection, preprocessing, tree
from sklearn.metrics import auc, confusion_matrix, roc_curve


## Get Data



In [ ]:
# Need xlrd to read XLS files
#df = pd.read_excel('http://biostat.mc.vanderbilt.edu/wiki/pub/Main/DataSets/titanic3.xls')
df = pd.read_excel('data/titanic3.xls')

In [ ]:
df

In [ ]:
df.dtypes

In [ ]:
df.describe()

## Exercise: Load Data
Load the Donner Party data.
http://www.stat.ufl.edu/~winner/datasets.html 
in a DataFrame named ``donner``

Source: Kristin Johnson's killer webpage @ www.utahcrossing.org 

Description: Statistics regarding the 89 members of Donner party
1846-1847.


Variables/Columns
* Name  1-27
* Age   29-30
* Gender   31
* Survive   46   /*  1=Yes, 0=No  */
* Date of death   49-58   /* mmddyyyy10.  */
* rescue party   67     /*  among survivors  */
* date joined party  72-81   /* mmddyyyy10. */
* trapped in mountains  90  /* 1=Yes, 0=no */
* camp  91-92     /* LC=Lake Camp, AC=Alden Creek  (mid december)  */

In [ ]:
!head data/donner.dat

In [ ]:
# Convert column oriented data to csv
with open('data/donner.csv', 'w') as fout:
    fout.write('name,age,gender,survive,death_date,rescue,party_date,mountains,camp\n')
    for line in open('data/donner.dat').read().split('\n'):
        items = [line[:28], line[28:30], line[30:31], line[45:46],
                 line[48:58], line[66:67], line[71:81],
                line[89:90], line[90:]]
        new_line = ','.join(x.strip() for x in items)+'\n'
        fout.write(new_line)


## Convert data
Most data need to be numeric


-  class: Passenger class (1 = first; 2 = second; 3 = third)
-  name: Name
-  sex: Sex
-  age: Age
-  sibsp: Number of siblings/spouses aboard
-  parch: Number of parents/children aboard
-  ticket: Ticket number
-  fare: Passenger fare
-  cabin: Cabin
-  embarked: Port of embarkation (C = Cherbourg; Q = Queenstown; S =
   Southampton)
-  boat: Lifeboat (if survived)
-  body: Body number (if did not survive and body was recovered)

In [ ]:
df.dtypes

In [ ]:
# Hint: use .value_counts() for objects (strings)
df.embarked.value_counts()

In [ ]:
df.cabin.value_counts()

In [ ]:
ignore = set('boat,body,home.dest,name,ticket'.split(','))
cols = [c for c in df.columns if c != 'survived' and c not in ignore]
X = df[cols]
y = df.survived

In [ ]:
X.dtypes

In [ ]:
dummy_cols = 'pclass,sex,cabin,embarked'.split(",")
df2 = pd.get_dummies(df, columns=dummy_cols)
cols = [c for c in df2.columns if c != 'survived' and c
         not in ignore and c not in dummy_cols]
X = df2[cols]
y = df2.survived

In [ ]:
X.dtypes

In [ ]:
# Check if we are numeric!
X.dtypes.value_counts()

In [ ]:
# Can't handle NaN
X.isnull()

In [ ]:
# Cols with missing data
X.isnull().any(axis=0)

In [ ]:
# Rows with missing data
X.isnull().any(axis=1)

In [ ]:
X[X.isnull().any(axis=1)]

In [ ]:
X.median()

In [ ]:
# Replace data with median
X = X.fillna(X.median())    

In [ ]:
# Any missing data?
X.isnull().any(axis=0).any()

## Exercise: Convert donner to numeric
* Make an X_donner dataframe, and y_donner series.
* We can use ``pd.to_datetime`` to create dates (or add ``parse_dates=`` to read_csv call). Then we can access the ``.dt`` property to get parts of the date (month, year, day)
* Drop name, death_date and party_date
* Create dummy columns for gender and camp
* Fill missing values with 0 (see ``.fill_na``)



## Decision Tree

In [ ]:
dt = tree.DecisionTreeClassifier(random_state=42)
dt.fit(X, y)

In [ ]:
dt.predict(X)

In [ ]:
dt.score(X, y)

In [ ]:
# Visualize (if you have graphviz)
# note that this goes to a Unix path
tree.export_graphviz(dt, out_file='tree1.dot', 
                     feature_names=X.columns, class_names=['Died', 'Survived'],
                    filled=True
                    )

In [ ]:
%%bash
# This doesn't run on Windows. Also requires that you have graphviz installed (not a Python module)
dot -Tpng -otree1.png tree1.dot

![Big Tree](tree1.png)

In [ ]:
print(sorted(zip(X.columns, dt.feature_importances_), key=lambda x:x[1], reverse=True))

## Exercise 
Create dt_donner model
* What are the most important features

## Testing/Traing Set & Try and Generalize the Model

In [ ]:
X_train, X_test, y_train, y_test = model_selection.\
    train_test_split(X, y, test_size=.3, random_state=42)

In [ ]:
dt2 = tree.DecisionTreeClassifier(random_state=42, max_depth=3)
dt2.fit(X_train, y_train)
dt2.score(X_test, y_test)

In [ ]:
tree.export_graphviz(dt2, out_file='tree2.dot', 
                     feature_names=X.columns, class_names=['Died', 'Survived'],
                    filled=True
                    )

In [ ]:
%%bash
dot -Tpng -otree2.png /tmp/tree2.dot


![Big Tree](tree2.png)

## Exercise: Decision Tree

* Create a testing and training set (``X_train_donner``, ``y_train_donner``...)
* Check if the model generalizes to the testing set
* Visualize the tree (if you have graphviz)

## Feature Engineering
Can try to add features to see if it improves performance

## (Bonus) Exercise - 
Try to add some derivitive columns to see if they help

## ROC Curve


In [ ]:

def fig_with_title(ax, title, figkwargs):
    if figkwargs is None:
        figkwargs = {}
    if not ax:
        fig = plt.figure(**figkwargs)
        ax = plt.subplot(111)
    else:
        fig = plt.gcf()
    if title:
        ax.set_title(title)
    return fig, ax

def plot_roc_curve_binary(clf, X, y, label='ROC Curve (area={area:.3})',
                          title="ROC Curve", pos_label=None, sample_weight=None,
                          ax=None, figkwargs=None):
    ax = ax or plt.subplot(111)
    ax.set_xlim([-.1, 1])
    ax.set_ylim([0, 1.1])
    y_score = clf.predict_proba(X)
    if y_score.shape[1] != 2 and not pos_label:
        warnings.warn("Shape is not binary {} and no pos_label".format(y_score.shape))
        return
    try:
        fpr, tpr, thresholds = roc_curve(y, y_score[:,1], pos_label=pos_label,
                                     sample_weight=sample_weight)
    except ValueError as e:
        if 'is not binary' in str(e):
            warnings.warn("Check if y is numeric")
            raise

    roc_auc = auc(fpr, tpr)
    fig, ax = fig_with_title(ax, title, figkwargs)

    ax.plot(fpr, tpr, label=label.format(area=roc_auc))
    ax.plot([0, 1], [0, 1], '--', color=(0.6, 0.6, 0.6), label='Guessing')
    ax.set_xlabel('False Positive Rate')
    ax.set_ylabel('True Positive Rate')
    ax.legend(loc="lower right")
    return fig, ax

In [ ]:
plot_roc_curve_binary(dt2, X_test, y_test, figkwargs={'figsize':(14,10)})

## Exercise - ROC Curve

Plot the ROC curve for the Donner party

## Random Forest!

In [ ]:
rf = ensemble.RandomForestClassifier(random_state=41)
rf.fit(X_train, y_train)
rf.score(X_test, y_test)

In [ ]:
plot_roc_curve_binary(rf, X_test, y_test, figkwargs={'figsize':(14,10)})

## Exercise - Random Forest
* Create ``rf_donner``
* Find the accuracy
* Plot ROC

## Confusion Matrix
A Confusion Matrix is another way to evaluate performance. You can see where false positives (lower left) and false negatives (upper right) are.

In [ ]:

def plot_confusion_matrix(clf, X, y, labels, random_state=42, annotate=True,
                          cmap=plt.cm.Blues,
                          title="Confusion Matrix", ax=None, figkwargs=None):
    fig, ax = fig_with_title(ax, title, figkwargs)
    #X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=random_state)
    y_pred = clf.predict(X)
    cm = confusion_matrix(y, y_pred)
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    fig.colorbar(im)
    ax.set_xticks(range(len(labels)))
    ax.set_xticklabels(labels, rotation=45)
    ax.set_yticks(range(len(labels)))
    ax.set_yticklabels(labels)
    ax.set_ylabel('True Label')
    ax.set_xlabel('Predicted Label')
    if annotate:
        for x in range(len(labels)):
            for y in range(len(labels)):
                plt.annotate(str(cm[x][y]),
                             xy=(y,x),
                             ha='center',va='center',color='red', fontsize=25, fontstyle='oblique')

    return fig, ax

In [ ]:
plot_confusion_matrix(rf, X_test, y_test, ['died', 'survived'])

## Exercise - Plot a Confusion Matrix

## Calibration Curves

from http://scikit-learn.org/stable/auto_examples/calibration/plot_calibration_curve.html

and https://jmetzen.github.io/2015-04-14/calibration.html

In [ ]:
from sklearn.calibration import CalibratedClassifierCV, calibration_curve
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (brier_score_loss, precision_score, recall_score,
                             f1_score)



def plot_calibration_curve(est, name, fig_index,                      
    X_train, X_test, y_train, y_test):
    """Plot calibration curve for est w/o and with calibration. """
    # Calibrated with isotonic calibration
    isotonic = CalibratedClassifierCV(est, cv=2, method='isotonic')

    # Calibrated with sigmoid calibration
    sigmoid = CalibratedClassifierCV(est, cv=2, method='sigmoid')

    # Logistic regression with no calibration as baseline
    lr = LogisticRegression(C=1., solver='lbfgs')

    fig = plt.figure(fig_index, figsize=(10, 10))
    ax1 = plt.subplot2grid((3, 1), (0, 0), rowspan=2)
    ax2 = plt.subplot2grid((3, 1), (2, 0))

    ax1.plot([0, 1], [0, 1], "k:", label="Perfectly calibrated")
    for clf, name in [(lr, 'Logistic'),
                      (est, name),
                      (isotonic, name + ' + Isotonic'),
                      (sigmoid, name + ' + Sigmoid')]:
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        if hasattr(clf, "predict_proba"):
            prob_pos = clf.predict_proba(X_test)[:, 1]
        else:  # use decision function
            prob_pos = clf.decision_function(X_test)
            prob_pos = \
                (prob_pos - prob_pos.min()) / (prob_pos.max() - prob_pos.min())

        clf_score = brier_score_loss(y_test, prob_pos, pos_label=y.max())
        print("%s:" % name)
        print("\tBrier: %1.3f" % (clf_score))
        print("\tPrecision: %1.3f" % precision_score(y_test, y_pred))
        print("\tRecall: %1.3f" % recall_score(y_test, y_pred))
        print("\tF1: %1.3f" % f1_score(y_test, y_pred))
        print("\tScore: %1.3f\n" % clf.score(X_test, y_test))

        fraction_of_positives, mean_predicted_value = \
            calibration_curve(y_test, prob_pos, n_bins=10)

        ax1.plot(mean_predicted_value, fraction_of_positives, "s-",
                 label="%s (%1.3f)" % (name, clf_score))

        ax2.hist(prob_pos, range=(0, 1), bins=10, label=name,
                 histtype="step", lw=2)

    ax1.set_ylabel("Fraction of positives")
    ax1.set_ylim([-0.05, 1.05])
    ax1.legend(loc="lower right")
    ax1.set_title('Calibration plots  (reliability curve)')

    ax2.set_xlabel("Mean predicted value")
    ax2.set_ylabel("Count")
    ax2.legend(loc="upper center", ncol=2)

    plt.tight_layout()
    
    
plot_calibration_curve(rf, 'Random Forest', 1,
    X_train, X_test, y_train, y_test)

## Exercise: Plot a Calibration Curve

## Optimizing Models
Models have hyperparameters that we can tune. Grid search cross validation will hold out some of the data for testing purposes, so we can pass in the full X and y into it.

In [ ]:
%%time
rf2 = ensemble.RandomForestClassifier()
params = {'max_features': [.4, 'auto'],
         'n_estimators': [15, 200, 500],
         'min_samples_leaf': [1, .1],
         'random_state':[42]}
cv = model_selection.GridSearchCV(rf2, params).fit(X_train, y_train)
print(cv.best_params_)

In [ ]:
rf3 = ensemble.RandomForestClassifier(**cv.best_params_)
rf3.fit(X_train, y_train)
rf3.score(X_test, y_test)

In [ ]:
rf4 = ensemble.RandomForestClassifier(random_state=42)
rf4.fit(X_train, y_train)
rf4.score(X_test, y_test)

## Exercise: Optimize Model
* Determine some hyper parameters that work better for the Donner set
* Create a new random forest and compare the score

## Learning Curves: Do we have enough data?
http://scikit-learn.org/stable/auto_examples/model_selection/plot_learning_curve.html




In [ ]:
def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=1, train_sizes=np.linspace(.1, 1.0, 5),
                       fig_opts=None):
    """
    Generate a simple plot of the test and training learning curve.

    Parameters
    ----------
    estimator : object type that implements the "fit" and "predict" methods
        An object of that type which is cloned for each validation.

    title : string
        Title for the chart.

    X : array-like, shape (n_samples, n_features)
        Training vector, where n_samples is the number of samples and
        n_features is the number of features.

    y : array-like, shape (n_samples) or (n_samples, n_features), optional
        Target relative to X for classification or regression;
        None for unsupervised learning.

    ylim : tuple, shape (ymin, ymax), optional
        Defines minimum and maximum yvalues plotted.

    cv : int, cross-validation generator or an iterable, optional
        Determines the cross-validation splitting strategy.
        Possible inputs for cv are:
          - None, to use the default 3-fold cross-validation,
          - integer, to specify the number of folds.
          - An object to be used as a cross-validation generator.
          - An iterable yielding train/test splits.

        For integer/None inputs, if ``y`` is binary or multiclass,
        :class:`StratifiedKFold` used. If the estimator is not a classifier
        or if ``y`` is neither binary nor multiclass, :class:`KFold` is used.

        Refer :ref:`User Guide <cross_validation>` for the various
        cross-validators that can be used here.

    n_jobs : integer, optional
        Number of jobs to run in parallel (default 1).
    """
    fig_opts = fig_opts or {}
    plt.figure(**fig_opts)
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = model_selection.learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    return plt

plot_learning_curve(rf3, 'Random Forest', X, y, fig_opts={'figsize':(14,10)})

## Exercise: Learning Curves
* Run a learning curve against the seed data? How much data do we need to train on?